In [3]:
import re
import numpy as np

In [ ]:
# load up all of the 19997 documents in the corpus
corpus = sc.textFile ("s3://chrisjermainebucket/comp330_A6/20_news_same_line.txt")

# each entry in validLines will be a line from the text file
validLines = corpus.filter(lambda x : 'id' in x)

# now we transform it into a bunch of (docID, text) pairs
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:]))

# now we split the text in each (docID, text) pair into a list of words
# after this, we have a data set with (docID, ["word1", "word2", "word3", ...])
# we have a bit of fancy regular expression stuff here to make sure that we do not
# die on some of the documents
regex = re.compile('[^a-zA-Z]')
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

# now get the top 20,000 words... first change (docID, ["word1", "word2", "word3", ...])
# to ("word1", 1) ("word2", 1)...
allWords = keyAndListOfWords.flatMap(lambda x: ((j, 1) for j in x[1]))

# now, count all of the words, giving us ("word1", 1433), ("word2", 3423423), etc.
allCounts = allWords.reduceByKey (lambda a, b: a + b)

# and get the top 20,000 words in a local array
# each entry is a ("word1", count) pair
topWords = allCounts.top (20000, lambda x : x[1])

# and we'll create a RDD that has a bunch of (word, dictNum) pairs
# start by creating an RDD that has the number 0 thru 20000
# 20000 is the number of words that will be in our dictionary
twentyK = sc.parallelize(range(20000))

# now, we transform (0), (1), (2), ... to ("mostcommonword", 1) ("nextmostcommon", 2), ...
# the number will be the spot in the dictionary used to tell us where the word is located
# HINT: make use of topWords in the lambda that you supply
dictionary = twentyK.map (lambda x:(topWords[x][0],x))

# finally, print out some of the dictionary, just for debugging
dictionary.top (10)

In [ ]:
# def conversion(tup, dict):
#     for x, y in tup:
#         dict.setdefault(x, []).append(y)
#     return dict

# dict = {}
# newDict = conversion(topWords, dict)

# newDict = dict(topWords)

# newDict
# {'the': [255140], 'to': [129353], 'of': [116303], 'a': [115484], 'and': [101867], 'i': [93333], 'in': [86847], 'is': [75498], 'that': [70756], 'ax': [62316], 'it': [58760], 'for': [49289], 'you': [48157], 's': [42335], 'from': [39653], 't': [36656], 'on': [35241], 'this': [34691], 'be': [32937], 'not': [32815], 'are': [32220], 'have': [31994], 'with': [30219], 'as': [27911], 'or': [25881], 'edu': [24776], 'was': [24243], 'if': [24232], 'm': [24012], 'but': [23241], 'they': [23203], 'subject': [21589], 'lines': [20891], 'date': [20778], 'doc': [20190], 'can': [20018], 'apr': [19679], 'by': [19545], 're': [18856], 'at': [18490]}

# newkeyAndList = dict(keyAndListOfWords)

# keyAndListOfWords.top(1)
# [('20_newsgroups/talk.religion.misc/84570', ['from', 'pharvey', 'quack', 'kfu', 'com', 'paul', 'harvey', 'subject', 're', 'i', 'll', 'see', 'your', 'demand', 'and', 'raise', 'you', 'was', 're', 'after', 'years', 'etc', 'date', 'apr', 'utc', 'lines', 'in', 'article', 'bil', 'okcforum', 'osrhe', 'edu', 'bill', 'conner', 'writes', 'keith', 'm', 'ryan', 'kmr', 'po', 'cwru', 'edu', 'wrote', 'mand', 'now', 'mdeep', 'thoughts', 'm', 'mby', 'jack', 'handey', 'm', 'mif', 'you', 'go', 'parachuting', 'and', 'your', 'parachute', 'doesn', 't', 'open', 'and', 'your', 'friends', 'are', 'all', 'watching', 'you', 'fall', 'i', 'think', 'a', 'funny', 'gag', 'would', 'be', 'to', 'pretend', 'you', 'were', 'swimming', 'm', 'keith', 'as', 'you', 'must', 'know', 'by', 'now', 'there', 'are', 'no', 'escape', 'sequences', 'here', 'ansi', 'or', 'otherwise', 'once', 'you', 'enter', 'here', 'your', 'terminal', 'beomes', 'dumb', 'there', 's', 'something', 'significant', 'about', 'all', 'this', 'you', 'are', 'in', 'the', 'village', 'many', 'happy', 'returns', 'be', 'seeing', 'you', 'your', 'ways', 'and', 'means', 'get', 'reign', 'of', 'the', 'tek', 'doc'])]

# newkeyAndList
# {'20_newsgroups/talk.religion.misc/84570': ['from', 'pharvey', 'quack', 'kfu', 'com', 'paul', 'harvey', 'subject', 're', 'i', 'll', 'see', 'your', 'demand', 'and', 'raise', 'you', 'was', 're', 'after', 'years', 'etc', 'date', 'apr', 'utc', 'lines', 'in', 'article', 'bil', 'okcforum', 'osrhe', 'edu', 'bill', 'conner', 'writes', 'keith', 'm', 'ryan', 'kmr', 'po', 'cwru', 'edu', 'wrote', 'mand', 'now', 'mdeep', 'thoughts', 'm', 'mby', 'jack', 'handey', 'm', 'mif', 'you', 'go', 'parachuting', 'and', 'your', 'parachute', 'doesn', 't', 'open', 'and', 'your', 'friends', 'are', 'all', 'watching', 'you', 'fall', 'i', 'think', 'a', 'funny', 'gag', 'would', 'be', 'to', 'pretend', 'you', 'were', 'swimming', 'm', 'keith', 'as', 'you', 'must', 'know', 'by', 'now', 'there', 'are', 'no', 'escape', 'sequences', 'here', 'ansi', 'or', 'otherwise', 'once', 'you', 'enter', 'here', 'your', 'terminal', 'beomes', 'dumb', 'there', 's', 'something', 'significant', 'about', 'all', 'this', 'you', 'are', 'in', 'the', 'village', 'many', 'happy', 'returns', 'be', 'seeing', 'you', 'your', 'ways', 'and', 'means', 'get', 'reign', 'of', 'the', 'tek', 'doc'], '20_newsgroups/talk.religion.misc/84569': ['date', 'tuesday', 'apr', 'edt', 'from', 'subject', 're', 'info', 'about', 'new', 'age', 'lines', 'in', 'article', 'qpalo', 'digi', 'lonestar', 'org', 'gerry', 'palo', 'the', 'danger', 'of', 'anti', 'cult', 'groups', 'is', 'that', 'while', 'they', 'can', 'expose', 'a', 'lot', 'of', 'deception', 'they', 'can', 'also', 'become', 'inquisitors', 'as', 'one', 'who', 'agrees', 'with', 'much', 'of', 'what', 'they', 'say', 'i', 'am', 'also', 'on', 'the', 'receiving', 'end', 'and', 'it', 'makes', 'me', 'realize', 'the', 'importance', 'of', 'respecting', 'the', 'freedom', 'of', 'belief', 'of', 'every', 'individual', 'and', 'also', 'of', 'not', 'jumping', 'to', 'conclusions', 'and', 'making', 'accusations', 'based', 'on', 'a', 'priori', 'assumptions', 'about', 'an', 'individual', 'or', 'group', 'for', 'my', 'money', 'the', 'primary', 'danger', 'of', 'anti', 'cult', 'groups', 'is', 'that', 'they', 'are', 'every', 'bit', 'as', 'wacky', 'as', 'the', 'groups', 'they', 'oppose', 'and', 'that', 'by', 'and', 'large', 'they', 'have', 'no', 'compunctions', 'about', 'printing', 'lies', 'half', 'truths', 'and', 'misleading', 'innuendos', 'as', 'part', 'of', 'their', 'exposes', 'a', 'recent', 'book', 'on', 'cults', 'i', 'picked', 'up', 'by', 'a', 'christian', 'author', 'quite', 'simply', 'mixed', 'in', 'all', 'non', 'christian', 'religions', 'except', 'the', 'jews', 'and', 'various', 'new', 'age', 'groups', 'with', 'various', 'fringe', 'groups', 'of', 'dubious', 'intent', 'and', 'legality', 'on', 'the', 'other', 'hand', 'the', 'watchman', 'fellowship', 'does', 'a', 'good', 'service', 'in', 'exposing', 'deceptive', 'practices', 'that', 'are', 'far', 'too', 'common', 'among', 'the', 'groups', 'they', 'monitor', 'given', 'the', 'record', 'of', 'american', 'christianity', 'any', 'group', 'that', 'falls', 'into', 'the', 'category', 'of', 'fundamentalist', 'or', 'born', 'again', 'is', 'automatically', 'into', 'the', 'inquisition', 'business', 'it', 'is', 'an', 'unavoidable', 'affliction', 'of', 'those', 'who', 'have', 'a', 'proprietary', 'license', 'on', 'the', 'truth', 'and', 'let', 's', 'not', 'forget', 'that', 'jonestown', 'and', 'the', 'branch', 'davidians', 'are', 'just', 'as', 'much', 'a', 'part', 'of', 'the', 'christian', 'tradition', 'as', 'the', 'missouri', 'synod', 'lutherans', 'and', 'may', 'in', 'fact', 'be', 'the', 'massadas', 'of', 'true', 'christian', 'believers', 'i', 'am', 'far', 'more', 'concerned', 'about', 'the', 'encroachment', 'of', 'overtly', 'christian', 'indoctrination', 'into', 'public', 'schools', 'than', 'i', 'am', 'about', 'yoga', 'classes', 'there', 'for', 'those', 'concerned', 'with', 'religious', 'freedom', 'without', 'a', 'selective', 'inquisitiorial', 'bent', 'people', 'for', 'the', 'american', 'way', 'p', 'o', 'box', 'washington', 'dc', 'americans', 'united', 'for', 'separation', 'of', 'church', 'state', 'fenton', 'street', 'silver', 'spring', 'md', 'jack', 'carroll', 'doc']}

# id = newkeyAndList.keys()
# list = newkeyAndList.values()

# ls = []
# for x in newkeyAndList:
#     eachList = newkeyAndList[x]
#     ls.add(eachList)
    
# for i in newDict:
#     for k in ls:
#         count_nonzero



task1

In [ ]:
newDict = dict(dictionary.top(20000))
def list2Array(ls, dic):
    word_count = np.zeros(20000)
    for word in ls:
        try:
            word_count[dic[word]] += 1
        except:
            continue
    return word_count
result = keyAndListOfWords.map(lambda x:(x[0], list2Array(x[1], newDict)))

task2

In [ ]:
doc_count = 19997
tf = result.map(lambda x: (x[0], x[1] / x[1].sum()))
idf_de = result.map(lambda x: ("idf_de", np.clip(x[1],0,1)))
idf_de = idf_de.reduceByKey(lambda a, b: a + b)
idf_de = idf_de.collect()
idf = np.log(doc_count/idf_de[0][1])
task2_result = tf.map(lambda x: (x[0], x[1] * idf))

task3

In [ ]:
def predictLabel(k, str):
...     str_word = regex.sub(' ', str).lower().split()
...     arr_word = list2Array(str_word, newDict)
...     tfidf = (arr_word / arr_word.sum()) * idf
...     Norm = task2_result.map(lambda x:(x[0], np.linalg.norm(x[1] - tfidf)))
...     kNN = Norm.top(k, lambda x: -x[1])
...     kNN2 = list(map(lambda x: (x[0][14:], x[1]), kNN))
...     kNN3 = list(map(lambda x: (x[0][:x[0].index("/")], x[1]), kNN2))
...     kNN4 = list(map(lambda x: x[0], kNN3))
...     clo = {}
...     for doc in kNN4:
...             try:
...                     clo[doc] += 1
...             except:
...                     clo[doc] = 1
...     return keyMax(clo)
... 
def keyMax(c):
...     v = list(c.values())
...     k = list(c.keys())
...     return k[v.index(max(v))]